# Load Libraries

In [30]:
import os
import sys
import sqlite3
import warnings
import numpy as np
import pandas as pd
import geopandas as gp
import plotly.express as px
import skill_metrics as sm
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import scipy.stats as scs
import seaborn as sns

import spei

sys.path.append(r'C:\Users\Pooya\w\GitHub\ShiraziPooya\DroughtMonitoringIran')

from app.utils.model_evaluator import ModelEvaluator


In [19]:
RESULTS_DIR = '../output/Stations'
os.makedirs(RESULTS_DIR, exist_ok=True)

# Load Data

In [24]:
DATABASE_PATH = "../database/database.db"

conn = sqlite3.connect(DATABASE_PATH)
df = pd.read_sql(sql='SELECT * FROM data', con=conn)
df['date'] = pd.to_datetime(df["date"], format="%Y-%m")
conn.close()

df

,region_id,region_name,station_name,station_id,lat,lon,station_elevation,date,tmax,tmin,...,ERA5_SPI_24,ERA5_SPEI_24,GPM_SPI_24,GPM_SPEI_24,PERSIANN_CDR_SPI_24,PERSIANN_CDR_SPEI_24,TRMM_SPI_24,TRMM_SPEI_24,TerraClimate_SPI_24,TerraClimate_SPEI_24
0,MASA,Mazandaran,Ramsar,40732,36.90,50.68,-20.0,1980-01-01,9.55,1.94,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MASA,Mazandaran,Ramsar,40732,36.90,50.68,-20.0,1980-02-01,8.46,2.48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MASA,Mazandaran,Ramsar,40732,36.90,50.68,-20.0,1980-03-01,10.53,5.66,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MASA,Mazandaran,Ramsar,40732,36.90,50.68,-20.0,1980-04-01,15.83,10.07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MASA,Mazandaran,Ramsar,40732,36.90,50.68,-20.0,1980-05-01,20.67,14.07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8230,MASA,Mazandaran,Alasht,99361,36.07,52.84,1805.0,2025-05-01,22.36,11.35,...,-0.960,NaN,-1.378,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8231,MASA,Mazandaran,Alasht,99361,36.07,52.84,1805.0,2025-06-01,20.64,11.66,...,-1.353,NaN,-1.689,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8232,MASA,Mazandaran,Alasht,99361,36.07,52.84,1805.0,2025-07-01,24.43,15.14,...,-0.881,NaN,-1.555,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8233,MASA,Mazandaran,Alasht,99361,36.07,52.84,1805.0,2025-08-01,26.52,16.82,...,-1.584,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Filter Data

In [27]:
# Select Stations:
# - Cluster 1 - Sari
# - Cluster 2 - Alasht
# - Cluster 3 - Ramsar

SELECTED_STATIONS = ["Sari", "Alasht", "Ramsar"]
START_DATE = "2006-09-01"
END_DATE = "2025-10-01"

selected_features = [
    'region_id',
    'region_name',
    'station_id',
    'station_name',
    'date',
    'rrr24',
    'GPM'
]

for di in ['SPI', 'SPEI']:
    for scale in [1, 3, 6, 9, 12, 15, 18, 21, 24]:
        selected_features.append(f'{di}_{scale}')
        selected_features.append(f'GPM_{di}_{scale}')

data = df[selected_features]\
    .query("station_name in @SELECTED_STATIONS and date >= @START_DATE and date < @END_DATE")



for di in ['SPEI', 'SPI']:
    for scale in [1, 3, 6, 9, 12, 15, 18, 21, 24]:
        data[f'{di}_{scale}_Class'] = pd.cut(data[f'{di}_{scale}'], bins=[-10, -2, -1.5, -1, 1, 1.5, 2, 10], labels=['ED', 'SD', 'MD', 'NN', 'MW', 'VW', 'EW'])
        data[f'{di}_{scale}_Class'] = data[f'{di}_{scale}_Class'].astype('category')

        data[f'GPM_{di}_{scale}_Class'] = pd.cut(data[f'GPM_{di}_{scale}'], bins=[-10, -2, -1.5, -1, 1, 1.5, 2, 10], labels=['ED', 'SD', 'MD', 'NN', 'MW', 'VW', 'EW'])
        data[f'GPM_{di}_{scale}_Class'] = data[f'GPM_{di}_{scale}_Class'].astype('category')

data     

,region_id,region_name,station_id,station_name,date,rrr24,GPM,SPI_1,GPM_SPI_1,SPI_3,...,SPI_12_Class,GPM_SPI_12_Class,SPI_15_Class,GPM_SPI_15_Class,SPI_18_Class,GPM_SPI_18_Class,SPI_21_Class,GPM_SPI_21_Class,SPI_24_Class,GPM_SPI_24_Class
320,MASA,Mazandaran,40732,Ramsar,2006-09-01,274.40,105.840,1.113,1.255,0.469,...,MD,NN,SD,MD,MD,MD,MD,NN,NN,NN
321,MASA,Mazandaran,40732,Ramsar,2006-10-01,285.52,118.296,0.346,0.057,0.428,...,NN,MD,MD,NN,NN,NN,NN,NN,NN,NN
322,MASA,Mazandaran,40732,Ramsar,2006-11-01,193.70,122.400,0.199,-0.090,0.606,...,NN,NN,MD,MD,NN,MD,NN,NN,NN,NN
323,MASA,Mazandaran,40732,Ramsar,2006-12-01,97.11,127.224,0.111,0.927,0.222,...,NN,NN,NN,NN,MD,NN,MD,NN,NN,NN
324,MASA,Mazandaran,40732,Ramsar,2007-01-01,64.52,43.896,-0.104,-1.562,0.038,...,NN,NN,NN,NN,MD,NN,NN,NN,NN,NN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8230,MASA,Mazandaran,99361,Alasht,2025-05-01,22.20,14.880,-0.311,-6.631,-1.337,...,MW,MD,NN,SD,NN,MD,NN,MD,NN,MD
8231,MASA,Mazandaran,99361,Alasht,2025-06-01,54.32,7.920,0.933,-0.766,-0.304,...,NN,ED,VW,MD,MW,MD,MW,NN,NN,SD
8232,MASA,Mazandaran,99361,Alasht,2025-07-01,19.20,13.392,-1.164,-0.278,-0.432,...,NN,ED,MW,MD,MW,MD,NN,SD,NN,SD
8233,MASA,Mazandaran,99361,Alasht,2025-08-01,16.00,NaN,-0.573,NaN,-0.407,...,NN,NaN,MW,NaN,NN,NaN,NN,NaN,NN,NaN


# Evaluator Ground Data

In [28]:
results_df = pd.DataFrame(
    columns=[
        'Station', 'Index', 'Scale', 'Accuracy', 'Precision', 'Recall', 'F1-score'
    ]
)


for st in data['station_name'].unique():
    df_st = data.query("station_name == @st")
    for di in ['SPEI', 'SPI']:
        for scale in [1, 3, 6, 9, 12, 15, 18, 21, 24]:
            
            df = df_st[[
                'region_id',
                'region_name',
                'station_id',
                'station_name',
                'date', 
                f'{di}_{scale}',
                f'GPM_{di}_{scale}',
                f'{di}_{scale}_Class', 
                f'GPM_{di}_{scale}_Class'
            ]].dropna()
            
            df.set_index('date', inplace=True)
            
            print(f"Station: {st}, Index: {di}, Scale: {scale}")
            report_dict = classification_report(
                y_true=df[f'{di}_{scale}_Class'],
                y_pred=df[f'GPM_{di}_{scale}_Class'],
                labels=['ED', 'SD', 'MD', 'NN', 'MW', 'VW', 'EW'],
                output_dict=True
            )
            
            results_df.loc[len(results_df)] = [
                st, di, scale,
                report_dict['accuracy'],
                report_dict['weighted avg']['precision'],
                report_dict['weighted avg']['recall'],
                report_dict['weighted avg']['f1-score']
            ]
            
            cm = confusion_matrix(
                y_true=df[f'{di}_{scale}_Class'], 
                y_pred=df[f'GPM_{di}_{scale}_Class'],
                labels=['ED', 'SD', 'MD', 'NN', 'MW', 'VW', 'EW']
            )
            
            plt.figure(figsize=(10, 8))
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['ED', 'SD', 'MD', 'NN', 'MW', 'VW', 'EW'], yticklabels=['ED', 'SD', 'MD', 'NN', 'MW', 'VW', 'EW'])
            plt.title(f'{st} - {di} {scale}-month')
            plt.ylabel('Actual')
            plt.xlabel('Predicted')
            fig_path = os.path.join(RESULTS_DIR, f'{st}-{di}-{scale} - Confusion Matrix.png')
            plt.savefig(fig_path, bbox_inches='tight')
            plt.close()            
            
            # Plot time series for station and GPM index
            f, ax = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
            spei.plot.si(df[f'{di}_{scale}'], ax=ax[0], cmap="vik_r")
            spei.plot.si(df[f'GPM_{di}_{scale}'], ax=ax[1], cmap="vik_r")
            ax[0].set_xlim(pd.to_datetime([START_DATE, END_DATE]))
            ax[0].set_title(f'{st} - {di} {scale}-month', fontsize=16)
            [x.grid() for x in ax]
            [ax[i].set_ylabel(n, fontsize=14) for i, n in enumerate([f'{di}{scale} (Station)', f'{di}{scale} (GPM)'])]
            plt.tight_layout()
            fig_path = os.path.join(RESULTS_DIR, f'{st}-{di}-{scale} - Timeseries.png')
            f.savefig(fig_path, bbox_inches='tight')
            plt.close(f)

results_df.to_csv(os.path.join(RESULTS_DIR, 'classification_results.csv'), index=False)


Station: Ramsar, Index: SPEI, Scale: 1
Station: Ramsar, Index: SPEI, Scale: 3
Station: Ramsar, Index: SPEI, Scale: 6
Station: Ramsar, Index: SPEI, Scale: 9
Station: Ramsar, Index: SPEI, Scale: 12
Station: Ramsar, Index: SPEI, Scale: 15
Station: Ramsar, Index: SPEI, Scale: 18
Station: Ramsar, Index: SPEI, Scale: 21
Station: Ramsar, Index: SPEI, Scale: 24
Station: Ramsar, Index: SPI, Scale: 1
Station: Ramsar, Index: SPI, Scale: 3
Station: Ramsar, Index: SPI, Scale: 6
Station: Ramsar, Index: SPI, Scale: 9
Station: Ramsar, Index: SPI, Scale: 12
Station: Ramsar, Index: SPI, Scale: 15
Station: Ramsar, Index: SPI, Scale: 18
Station: Ramsar, Index: SPI, Scale: 21
Station: Ramsar, Index: SPI, Scale: 24
Station: Sari, Index: SPEI, Scale: 1
Station: Sari, Index: SPEI, Scale: 3
Station: Sari, Index: SPEI, Scale: 6
Station: Sari, Index: SPEI, Scale: 9
Station: Sari, Index: SPEI, Scale: 12
Station: Sari, Index: SPEI, Scale: 15
Station: Sari, Index: SPEI, Scale: 18
Station: Sari, Index: SPEI, Scale: 

In [35]:
for st in SELECTED_STATIONS:
    for di in ['SPEI', 'SPI']:
        for scale in [1, 3, 6, 9, 12, 15, 18, 21, 24]:
            print(f"\nMetrics for Station: {st}, Index: {di}, Scale: {scale}")
            ModelEvaluator(
                data.query('station_name == @st'),
                f'{di}_{scale}',
                f'GPM_{di}_{scale}'
            ).display_metrics()


Metrics for Station: Sari, Index: SPEI, Scale: 1
Model Performance Metrics:
Mean Absolute Error (MAE): 0.4157
Mean Squared Error (MSE): 0.2985
Root Mean Squared Error (RMSE): 0.5464
R-squared (R2): 0.7003
Explained Variance Score: 0.7004
Willmott's Index of Agreement (WI): 0.7517
Nash–Sutcliffe Efficiency (NS): 0.7003
Mean Absolute Deviation (MAD): 0.4157

Metrics for Station: Sari, Index: SPEI, Scale: 3
Model Performance Metrics:
Mean Absolute Error (MAE): 0.4595
Mean Squared Error (MSE): 0.3520
Root Mean Squared Error (RMSE): 0.5933
R-squared (R2): 0.6240
Explained Variance Score: 0.6251
Willmott's Index of Agreement (WI): 0.7140
Nash–Sutcliffe Efficiency (NS): 0.6240
Mean Absolute Deviation (MAD): 0.4595

Metrics for Station: Sari, Index: SPEI, Scale: 6
Model Performance Metrics:
Mean Absolute Error (MAE): 0.4441
Mean Squared Error (MSE): 0.4596
Root Mean Squared Error (RMSE): 0.6779
R-squared (R2): 0.6144
Explained Variance Score: 0.6145
Willmott's Index of Agreement (WI): 0.7304


In [40]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

di = "SPI"
scale = 24

x=f'{di}_{scale}'
y=f'GPM_{di}_{scale}'

for st in SELECTED_STATIONS:
    station_data = data.query("station_name == @st")[['date', x, y]].dropna()

    fig = px.scatter(
        station_data,
        x=x,
        y=y,
        labels={
            x: f"{st}<br>{di}{scale} (Station)",
            y: f'{di}{scale} (GPM)'
        },
        opacity=0.7,
        color_discrete_sequence=["black"],  # make points black
    )

    # Make scatter markers bigger
    fig.update_traces(marker=dict(size=8, color="black"))

    # Equal axis limits
    max_limit = max(station_data[x].max(),
                    station_data[y].max()) + 0.5
    min_limit = min(station_data[x].min(),
                    station_data[y].min()) - 0.5
    
    # Equal ticks
    tick_vals = np.arange(min_limit, max_limit, 1)

    fig.update_layout(
        xaxis=dict(range=[min_limit, max_limit]),
        yaxis=dict(range=[min_limit, max_limit]),
        width=600,
        height=600,
        title=None,
        margin=dict(l=20, r=20, t=20, b=20),
    )
    fig.update_yaxes(scaleanchor="x", scaleratio=1)

    # Add 1:1 line (hidden from legend)
    fig.add_trace(go.Scatter(
        x=[min_limit, max_limit],
        y=[min_limit, max_limit],
        mode="lines",
        line=dict(color="red", dash="dash"),
        showlegend=False
    ))

    fig.show()
